### IMPORT LIBRARY

In [1]:
import pandas as pd #for manipulating the csv data
pd.set_option('display.max_rows', None)
import numpy as np #for mathematical calculation
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from math import log2, e, log, exp,log
import sys, threading
from sklearn.impute import KNNImputer
sys.setrecursionlimit(10**7) # max depth of recursion
threading.stack_size(2**27)  # new thread will get stack of such size

0

### FUNGSI PRA PROSES

In [2]:
def nulling (df_in, col_name):
    for i,row in enumerate(df_in[col_name]):
        if row == 0:
            df_in.at[i,col_name]=None
            
    df_out = df_in
    return df_out

def rounding (df_in, col_name,r):
    for i,row in enumerate(df_in[col_name]):
        df_in.at[i,col_name]=round(row,r)
    df_out = df_in
    return df_out

def impute_knn(df_in):
    imputer = KNNImputer(n_neighbors=3)
    imputed = imputer.fit_transform(df_in)
    df_imputed = pd.DataFrame(imputed, columns=df_in.columns)
    return df_imputed

def impute_knnn(df_in):
    data = df_in
    imputer = KNNImputer(n_neighbors=3)
    #class 0
    class0 = data[data['Outcome']==0]
    class0idx = class0.index
    #class 1
    class1=data[data['Outcome']==1]
    class1idx = class1.index

    imputedc0 = imputer.fit_transform(class0)
    imputedc1 = imputer.fit_transform(class1)
    df_imputed0 = pd.DataFrame(imputedc0,index=class0idx, columns=data.columns)
    df_imputed1 = pd.DataFrame(imputedc1,index=class1idx, columns=data.columns)

    df_imputed=pd.concat([df_imputed0,df_imputed1])
    df_imputed=df_imputed.sort_index()
    return df_imputed
    
def praproses(daf,knn=1):
    df=daf.copy()
    tempn= ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
    tempr={
        'Glucose':0,
        'Pregnancies':0,
        'BloodPressure':0,
        'SkinThickness':0,
        'Insulin':0,
        'BMI':1
    }
    if knn == 1:
        for i in df.head():
            if i in tempn:
                df= nulling(df,i)
        df=impute_knnn(df)

    for i in df.head():
        if i in tempr:
            df=rounding(df,i,tempr[i])
    return df

def get_label_v(labels):
    n_labels = len(labels)
    if n_labels <= 1:
        return 0
    value,counts = np.unique(labels, return_counts=True)
    probs = counts / n_labels
    
    return value,counts,n_labels,probs

def entropy(probs):
    ent = 0.
    try:
        for i in probs:
            ent -= i * log2(i)
        return ent
    except:
        ent=0
        return ent

def ign_zero(x,y):
    try:
        return x/y
    except:
        return 0

def selecter(val,df,col):
    sk=len(df.loc[df[col]<=val])
    sl=len(df.loc[df[col]>val])
    sk0=len(df.loc[(df[col]<=val)&(df['Outcome']==0)].filter(col))
    sk1=len(df.loc[(df[col]<=val)&(df['Outcome']==1)].filter(col))
    sl0=len(df.loc[(df[col]>val)&(df['Outcome']==0)].filter(col))
    sl1=len(df.loc[(df[col]>val)&(df['Outcome']==1)].filter(col))
    pskt=[ign_zero(sk0,sk),ign_zero(sk1,sk)]
    pslt=[ign_zero(sl0,sl),ign_zero(sl1,sl)]
    
    return sk,sk0,sk1,sl,sl0,sl1,pskt,pslt

def get_max(df):
    minval =-1
    val=0
    for i in df:
        if i[1] > minval:
            minval=i[1]
            val=i[0]
    return minval,val
    
def diskritisasi(df,l_data,l_col):
    temp=[]
    value,counts,n_val,probs = get_label_v(df[l_col])
    for idx,i in enumerate(value):
        sk,_,_,sl,_,_,pskt,pslt = selecter(i,df,l_col)
        epskt=entropy(pskt)
        epslt=entropy(pslt)
        _,_,n_data,pdata= get_label_v(df[l_data])
        entt=entropy(pdata)
        informa= entt-(sk/n_data*epskt+sl/n_data*epslt)
        temp.append([i,informa])
    return get_max(temp)

def helper_disk(df,l_data):
    temp={}
    for i in df.head():
        if i != l_data:
            temp[i]=diskritisasi(df,l_data,i)
    return temp    

def tosplt(df,sp):
    zz=df
    for i in sp:
        aa= [sp[i][1] if x== True else sp[i][1]+1 for x in zz[i]<=sp[i][1] ]
        zz.drop(columns=[i])
        zz[i]=aa
    return zz

def runAll(dataa):
    data=dataa.copy()
    data=praproses(data)
    splitter = helper_disk(data,'Outcome')
    print(splitter)
    zz=data.copy()    
    pp_disc=tosplt(zz,splitter)

    return pp_disc

def runNoknn(dataa):
    data=dataa.copy()
    data=praproses(data,0)
    splitter = helper_disk(data,'Outcome')
    zz=data.copy()    
    pp_disc=tosplt(zz,splitter)

    return pp_disc

### INISIALISASI DATASET DAN SKENARIO DATASET

#### DATASET ASLI

In [3]:
raw_data = pd.read_csv('diabetes.csv')
raw_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


#### DATASET SETELAH IMPUTASI

In [4]:
datasets_imputed = praproses(raw_data)
datasets_imputed

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,134.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,67.0,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,30.0,208.0,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0
5,5.0,116.0,74.0,18.0,109.0,25.6,0.201,30.0,0.0
6,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26.0,1.0
7,10.0,115.0,75.0,35.0,118.0,35.3,0.134,29.0,0.0
8,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53.0,1.0
9,8.0,125.0,96.0,32.0,145.0,38.7,0.232,54.0,1.0


#### DATASET SETELAH DISKRITISASI TANPA IMPUTASI KNN

In [5]:
dnoknn = runNoknn(raw_data)
dnoknn

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,128,69,32,120,28.8,1.527,29,1
1,6,127,68,31,120,27.8,0.527,29,0
2,7,128,68,31,120,27.8,1.527,29,1
3,6,127,68,31,120,28.8,0.527,28,0
4,6,128,68,32,121,28.8,1.527,29,1
5,6,127,69,31,120,27.8,0.527,29,0
6,6,127,68,32,120,28.8,0.527,28,1
7,7,127,68,31,120,28.8,0.527,29,0
8,6,128,69,32,121,28.8,0.527,29,1
9,7,127,69,31,120,27.8,0.527,29,1


#### DATASET SETELAH DISKRITISASI DENGAN IMPUTASI KNN

In [6]:
dwithknn = runAll(raw_data)
dwithknn

{'Pregnancies': (0.039180260954807755, 6.0), 'Glucose': (0.13518841941127935, 127.0), 'BloodPressure': (0.018731718578645196, 63.0), 'SkinThickness': (0.07250945002996534, 23.0), 'Insulin': (0.14435516834581175, 108.0), 'BMI': (0.07469977982812326, 27.8), 'DiabetesPedigreeFunction': (0.02079639637723707, 0.527), 'Age': (0.0724727100056407, 28.0)}


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,128.0,64.0,24.0,109.0,28.8,1.527,29.0,1.0
1,6.0,127.0,64.0,24.0,108.0,27.8,0.527,29.0,0.0
2,7.0,128.0,64.0,24.0,109.0,27.8,1.527,29.0,1.0
3,6.0,127.0,64.0,23.0,108.0,28.8,0.527,28.0,0.0
4,6.0,128.0,63.0,24.0,109.0,28.8,1.527,29.0,1.0
5,6.0,127.0,64.0,23.0,109.0,27.8,0.527,29.0,0.0
6,6.0,127.0,63.0,24.0,108.0,28.8,0.527,28.0,1.0
7,7.0,127.0,64.0,24.0,109.0,28.8,0.527,29.0,0.0
8,6.0,128.0,64.0,24.0,109.0,28.8,0.527,29.0,1.0
9,7.0,127.0,64.0,24.0,109.0,28.8,0.527,29.0,1.0


### FUNGSI ALGORITMA C4.5

In [7]:
def calc_total_entropy(train_data, label, class_list,t):
    total_row = train_data[t].sum() #the total size of the dataset
    total_entr = 0
    
    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].sum()[t] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset
    
    return total_entr

def calc_entropy(feature_value_data, label, class_list,t):
    class_count = feature_value_data[t].sum()
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].sum()[t] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy


def calc_info_gain(feature_name, train_data, label, class_list,t):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data[t].sum()
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data[t].sum()
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list,t) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calc_total_entropy(train_data, label, class_list,t) - feature_info #calculating information gain by subtracting


def calc_sp_info(feature_name, train_data, label, class_list,t):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data[t].sum()
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        feature_value_count = feature_value_data[t].sum()
        #feature_value_entropy = calc_entropy(feature_value_data, label, class_list)
        feature_value_probability = feature_value_count/total_row
        feature_info += -feature_value_probability * np.log2(feature_value_probability)
        
    return feature_info


def find_most_informative_feature(train_data, label, class_list,t):
    feature_list = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction','Age']

    max_gn_rat = -1
    gn_rat = -1
    max_info_feature = None
    
    for feature in feature_list:  
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list,t)
        feature_sp_info = calc_sp_info(feature, train_data, label, class_list,t)
        try:
            gn_rat=feature_info_gain.item()/feature_sp_info.item()
        except:
            gn_rat = -1
        if max_gn_rat < gn_rat:
            max_gn_rat = gn_rat
            max_info_feature = feature
    return max_info_feature


def generate_sub_tree(feature_name, train_data, label, class_list, t,cek=None):
    if feature_name==None:
        feature_name=cek
    xcv={}
    for tmp in train_data[feature_name].unique():
        ss=train_data[train_data[feature_name]==tmp].sum()[t]
        xcv[tmp]=ss

    feature_value_count_dict= pd.Series(xcv)
    f_left=8
    l_node=False

    cf= ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction','Age']
    for f in cf:
        panj = len(train_data[f].unique())
        if panj == 1:
            f_left-=1
    if f_left<=1:
        l_node=True
    tree = {} #sub tree or node
    cek_fitur = cek

    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        cek_count = []
        cek_class = []

        for cek in class_list:
            class_count = feature_value_data[feature_value_data[label] == cek].sum()[t] #count of class c
            cek_count.append(class_count)
            cek_class.append(cek)

        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].sum()[t] #count of class c

            if l_node ==True:
                if cek_count[1] > cek_count[0] :
                    tree[feature_value] = float(cek_class[1]) #adding node to the tree
                    train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                    assigned_to_node = True
                else : 
                    tree[feature_value] = float(cek_class[0]) #adding node to the tree
                    train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                    assigned_to_node = True
            elif cek_fitur != feature_name : #tambahan pengecekan
                if class_count == count: #count of feature_value = count of class (pure class)
                    tree[feature_value] = c #adding node to the tree
                    train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                    assigned_to_node = True
            elif cek_fitur == feature_name : #tambahan pengecekan
                if cek_count[1] > cek_count[0] :
                    tree[feature_value] = float(cek_class[1]) #adding node to the tree
                    train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                    assigned_to_node = True
                else : 
                    tree[feature_value] = float(cek_class[0]) #adding node to the tree
                    train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                    assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #should extend the node, so the branch is marked with ?

    return tree, train_data

def make_tree(root, prev_feature_value, train_data, label, class_list,t , cek_fitur=None):

    if train_data[t].sum() != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list,t) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list,t, cek_fitur)
        next_root = None
        if max_info_feature == None :
            max_info_feature=cek_fitur
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                cek_fitur = max_info_feature
                make_tree(next_root, node, feature_value_data, label, class_list,t, cek_fitur) #recursive call with updated dataset



def generate_sub_stump(feature_name, train_data, label, class_list, t,cek=None):
    if feature_name==None:
        feature_name=cek

    xcv={}
    for tmp in train_data[feature_name].unique():
        ss=train_data[train_data[feature_name]==tmp].sum()[t]
        xcv[tmp]=ss

    feature_value_count_dict= pd.Series(xcv)
    f_left=8
    l_node=False

    cf= ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction','Age']
    for f in cf:
        panj = len(train_data[f].unique())
        if panj == 1:
            f_left-=1
    if f_left<=1:
        l_node=True
    tree = {}


    for feature_value, count in feature_value_count_dict.iteritems():

        feature_value_data = train_data[train_data[feature_name] == feature_value] 

        cek_count = []
        cek_class = []

        for cek in class_list:
            class_count = feature_value_data[feature_value_data[label] == cek].sum()[t] #count of class c
            cek_count.append(class_count)
            cek_class.append(cek)

        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].sum()[t]
            if cek_count[1] > cek_count[0] :
                    tree[feature_value] = float(cek_class[1]) 
            else : 
                tree[feature_value] = float(cek_class[0])
  
    return tree

def make_stump(root, prev_feature_value, train_data, label, class_list,t , cek_fitur=None):
    if train_data[t].sum() != 0:
        max_info_feature = find_most_informative_feature(train_data, label, class_list,t) #most informative feature
        tree = generate_sub_stump(max_info_feature, train_data, label, class_list,t, cek_fitur)
        root[max_info_feature] = tree

def c45(train_data_m, label,t):
    train_data = train_data_m.copy() #getting a copy of the dataset
    if type(t) == int :
        train_data,tn= weigher(train_data,t)
    else :
        tn=t
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list,tn) #start calling recursion
    return tree

def s_c45(train_data_m, label,t):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    #print("class list :", class_list)
    make_stump(tree, None, train_data_m, label, class_list,t) #start calling recursion
    return tree

def formatData(t,s):
    if not isinstance(t,dict) and not isinstance(t,list):
        print("\t"*s+str(t))
    else:
        for key in t:
            print("\t"*s+str(key))
            if not isinstance(t,list):
                formatData(t[key],s+1)


def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

def evaluate(tree, test_data_m):
    hasil_prediksi = []
    for i in range (len(test_data_m)): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[i]) #predict the row
        hasil_prediksi.append(result) #mendapatkan hasil prediksi dari data yang diuji

    return hasil_prediksi

### FUNGSI AKURASI

In [8]:
def performa(y_test, y_pred):
    #confusion matrix
    matrix = confusion_matrix(y_test, y_pred, labels=[1,0])
    print('Confusion matrix : \n',matrix)
    print("")

    tp, fn, fp, tn = confusion_matrix(y_test, y_pred,labels=[1,0]).reshape(-1)
    print("True positive :", tp)
    print("False negative :", fn)
    print("False positive :", fp)
    print("True negative :", tn)
    print("")

    matrix = classification_report(y_test, y_pred,labels=[1,0])
    print('Classification report : \n',matrix)
    print("")

    #akurasi
    accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
    presisi = round((tp/(tp+fp))*100, 2)
    recall = round((tp/(tp+fn))*100, 2)
    print("Akurasi fold ini adalah :", accuracy, " %")
    print("Presisi fold ini adalah :", presisi, " %")
    print("Recall fold ini adalah :", recall, " %")
    accuracy2 = round(((tp+tn)/(tp+fn+fp+tn)*100), 2)
    print("===========================================================")

    return accuracy, presisi, recall

### FUNGSI ADABOOST

In [9]:
def t_to(t):
    t=str(t)
    t='w'+t
    return t


def weigher(train_data,t,missc=None,alpha=None):
    ts=t_to(t)
    tsb=t_to(t-1)
    val = train_data.shape[0]
    if t == 0:
        zz = [1/val for _ in range(val)]
        train_data[ts]=zz
        return train_data,ts
    else :
        xx=[x for x in train_data[tsb]]
        train_data[ts]=xx
        for i in missc:
            nweight=train_data[ts][i]*exp(alpha)
            train_data.at[i,ts]=nweight
        norm = [x/train_data[ts].sum() for x in train_data[ts]]
        train_data[ts]=norm
        train_data=train_data.drop(columns=[tsb])
        
        return train_data,ts

def stump(tree,train_data,t):
    hp = evaluate(tree,train_data)
    missc=[]
    bennc=[]
    erate=0
    train_data['hp']=hp
    for i in train_data.index:
        if train_data['Outcome'][i] != train_data['hp'][i]:
            missc.append(i)
            erate+=train_data[t][i]
        else :
            bennc.append(i)
    alpha = log((1-erate)/erate)
    return missc,alpha

def adaboost(t,train_data):
    missc=0
    alpha=0
    hk=[]
    for i in range (t):
        train_data,tn = weigher(train_data,i,missc,alpha)
        tree=s_c45(train_data,'Outcome',tn)
        missc,alpha=stump(tree,train_data,tn)
        print(tn,tree,alpha)
        hk.append([tree,alpha])
    return hk

def predict_adb(tree,test_data_m,sk):
    pred=[]
    w_pred=[]
    for ij in range(len(test_data_m)):
        ppos=0
        pneg=0
        ttmp=[]
        for ji in range(sk):
            hk=predict(tree[ji][0],test_data_m.iloc[ij])
            ttmp.append(hk)
            if hk == 0:
                pneg+=tree[ji][1]
            elif hk == 1:
                ppos+=tree[ji][1]
        if ppos>pneg :
            hk= 1 
        elif ppos<pneg:
            hk = 0
        w_pred.append(ttmp)
        pred.append(hk)
    return pred

### FUNGSI KFOLD

In [10]:
def k_fold(dataset,t,skn):
    polo=len(dataset)
    k=10
    fsize=round(polo/k)
    c45_final={}
    c45_final_acc =[]
    c45_final_pre=[]
    c45_final_rec=[]
    adab_final={}
    adab_final_acc=[]
    adab_final_pre=[]
    adab_final_rec=[]
    adatree=[]
    treec45s=[]

    for o in range(k):
        ada_sk=[]
        ada_pre=[]
        ada_rec=[]
        bak=round(polo/k)
        aa= 0 if o==0 else o+1
        bak= bak if aa==0 else bak*aa
        baw=bak-fsize 
        test_data = dataset[aa:bak] if aa == 0 else dataset[baw:bak]
        train_data = dataset[bak:] if aa == 0 else pd.concat([dataset[:baw],dataset[bak:]])

        y_test=test_data['Outcome']
        yyxx=[x for x in y_test]
        
        print(">>> FOLD KE ",o+1)
        print("Panjang data test :", test_data.shape[0])
        print("Panjang data train :", train_data.shape[0])
        print("Data Test :\n",yyxx)
        
        print("*******************")
        print("Algoritma C4.5")
        print("*******************")
        treec45=c45(train_data.copy(),'Outcome',0)

        c45_y_pred =evaluate(treec45,test_data)
        
        print("Hasil Prediksi C45 :\n", c45_y_pred)
        
        accuracy, presisi, recall = performa(y_test, c45_y_pred)
        c45_final_acc.append(accuracy)
        c45_final_pre.append(presisi)
        c45_final_rec.append(recall)

        treec45s.append(treec45)
        
        adahk=adaboost(t,train_data.copy())
        hsk=[]
        for sk in skn:
            hkk=predict_adb(adahk,test_data,sk)
            hsk.append(hkk)
            print("*******************")
            print("Adaboost ","t-",sk)
            print("*******************")
            print("Hasil Prediksi Adaboost :\n", hkk)
            ax1,ax2,ax3 = performa(y_test, hkk)
            ada_sk.append(ax1)
            ada_pre.append(ax2)
            ada_rec.append(ax3)

        adatree.append(adahk)
        adab_final_acc.append(ada_sk)
        adab_final_pre.append(ada_pre)
        adab_final_rec.append(ada_rec)
    
    c45_final['akurasi']=c45_final_acc
    c45_final['presisi']=c45_final_pre
    c45_final['recall']=c45_final_rec

    adab_final['akurasi']=adab_final_acc
    adab_final['presisi']=adab_final_pre
    adab_final['recall']=adab_final_rec
    
    return c45_final,adab_final,adatree,treec45s

### MENJALANKAN UJI SKENARIO

#### Dataset dengan imputasi K-NN

In [11]:
sknar=[5,10,15,20]
t=20
wk_acc_c45,wk_acc_adb,adatree,treec45s=k_fold(dwithknn,t,sknar)

>>> FOLD KE  1
Panjang data test : 77
Panjang data train : 691
Data Test :
 [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
*******************
Algoritma C4.5
*******************
Hasil Prediksi C45 :
 [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
Confusion matrix : 
 [[15 17]
 [11 34]]

True positive : 15
False negative 

#### Dataset tanpa imputasi K-NN

In [12]:
nk_acc_c45,nk_acc_adb,adatree,treec45s=k_fold(dnoknn,20,sknar)

>>> FOLD KE  1
Panjang data test : 77
Panjang data train : 691
Data Test :
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
*******************
Algoritma C4.5
*******************
Hasil Prediksi C45 :
 [1.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1, 0.0, 1.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1, 0.0, 1, 0.0, 0.0, 1.0, 0.0, 0.0, 1, 0, 0, 0.0, 1.0, 1.0, 0, 1, 0.0, 1.0, 0.0, 0, 1.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0, 0.0]
Confusion matrix : 
 [[13 19]
 [ 9 36]]

True positive : 13
False negative : 19
False positive : 9
True negative : 36

Classification report : 
               precision    recall  f1-score   support

           1       0.59      0.41      0.48        32
           0 

### FUNGSI UNTUK MENAMPILKAN HASIL

In [13]:
def get_colname(df):
    tempp=df.columns
    return tempp[0][4:]
    
def get_max_at_fold(df):
    print("====================================")
    print("Nilai Max keseluruhan fold :")
    print("====================================")
    sr = pd.Series([df.iloc[i].max() for i in df.index],index=[i+1 for i in df.index])
    print(sr)
    print("\n>>> Nilai maksimal keseluruhan fold :")
    print(sr.idxmax(), "=",sr.max())

def get_mean_max_at_fold(df):
    print("====================================")
    print("Nilai rata-rata keseluruhan fold :")
    print("====================================")
    sr = pd.Series([df[i].mean() for i in df.columns],index=[i for i in df.columns])
    print(sr)
    print("\n>>> Nilai maksimal rata-rata keseluruhan fold :")
    print(sr.idxmax(), "=",sr.max())
    
def conclute_to_df(acc_adb, acc_c45):
    k = ["k"+str(i+1) for i in range(10)]
    sax = pd.DataFrame({'k': k})
    for i in acc_c45:
        idc45 = 'C45-'+str(i)
        sax[idc45] = acc_c45[i]
        for zz,skz in enumerate(sknar):
            adin='t-'+str(skz)+' '+str(i)
            sax[adin]=[x[zz] for x in acc_adb[i]]

    # sax.to_csv('noknn9.csv', index=False)
    return sax
    
def get_df(sax,pat):
    temp=[]
    for i in pat:
        col_aku = sax.columns[sax.columns.str.contains(i)].tolist() 
        temp.append(sax.get(col_aku))

    return temp

def get_conclute_data(acc_adb,acc_c45):
    sax = conclute_to_df(acc_adb,acc_c45)
    temp = get_df(sax,acc_adb.keys())
    for i in temp :
        i_name=get_colname(i)
        print("******************")
        print("Nilai",i_name)
        print("******************")
        print(i)
        get_max_at_fold(i)
        get_mean_max_at_fold(i)

### MENAMPILKAN HASIL

#### Dengan imputasi K-NN

In [14]:
get_conclute_data(wk_acc_adb,wk_acc_c45)

******************
Nilai akurasi
******************
   C45-akurasi  t-5 akurasi  t-10 akurasi  t-15 akurasi  t-20 akurasi
0        63.64        64.94         70.13         70.13         70.13
1        74.03        74.03         77.92         77.92         77.92
2        81.82        76.62         77.92         79.22         79.22
3        67.53        66.23         70.13         67.53         70.13
4        77.92        77.92         79.22         79.22         80.52
5        81.82        77.92         81.82         81.82         81.82
6        77.92        79.22         81.82         81.82         81.82
7        81.82        84.42         85.71         85.71         85.71
8        70.13        71.43         72.73         72.73         72.73
9        81.33        81.33         78.67         78.67         78.67
Nilai Max keseluruhan fold :
1     70.13
2     77.92
3     81.82
4     70.13
5     80.52
6     81.82
7     81.82
8     85.71
9     72.73
10    81.33
dtype: float64

>>> Nilai mak

#### Tanpa imputasi K-NN

In [15]:
get_conclute_data(nk_acc_adb,nk_acc_c45)

******************
Nilai akurasi
******************
   C45-akurasi  t-5 akurasi  t-10 akurasi  t-15 akurasi  t-20 akurasi
0        63.64        63.64         71.43         71.43         71.43
1        71.43        76.62         77.92         77.92         77.92
2        74.03        76.62         79.22         79.22         79.22
3        62.34        66.23         67.53         67.53         67.53
4        76.62        76.62         79.22         79.22         79.22
5        75.32        77.92         79.22         80.52         81.82
6        72.73        76.62         74.03         75.32         75.32
7        80.52        81.82         88.31         88.31         88.31
8        72.73        72.73         72.73         74.03         74.03
9        77.33        81.33         80.00         80.00         80.00
Nilai Max keseluruhan fold :
1     71.43
2     77.92
3     79.22
4     67.53
5     79.22
6     81.82
7     76.62
8     88.31
9     74.03
10    81.33
dtype: float64

>>> Nilai mak

### LAMPIRAN

#### Contoh Hasil Rule C-45 pada Fold - 1

{'Glucose': {127: {'Age': {28: {'Pregnancies': {6: {'BMI': {28.8: {'DiabetesPedigreeFunction': {0.527: {'Insulin': {120: {'BloodPressure': {69: {'SkinThickness': {32: 0.0,
                31: 0.0}},
              68: {'SkinThickness': {31: 0.0, 32: 0.0}}}},
            121: {'BloodPressure': {69: 0,
              68: {'SkinThickness': {32: 0.0, 31: 0.0}}}}}},
          1.5270000000000001: {'SkinThickness': {31: {'Insulin': {121: 0,
              120: {'BloodPressure': {68: 0.0, 69: 0.0}}}},
            32: {'BloodPressure': {68: {'Insulin': {120: 0.0, 121: 1.0}},
              69: 0}}}}}},
        27.8: {'DiabetesPedigreeFunction': {0.527: 0,
          1.5270000000000001: {'BloodPressure': {68: {'Insulin': {120: 0.0,
              121: 0.0}},
            69: 0}}}}}},
      7: 1}},
    29: {'Insulin': {120: {'BMI': {28.8: {'DiabetesPedigreeFunction': {1.5270000000000001: {'Pregnancies': {7: {'SkinThickness': {31: {'BloodPressure': {69: 0.0,
                68: 0.0}},
              32: {'SkinThickness': {32: 1.0}}}},
            6: {'BloodPressure': {69: {'SkinThickness': {31: 0.0, 32: 0.0}},
              68: {'SkinThickness': {32: 0.0, 31: 0.0}}}}}},
          0.527: {'SkinThickness': {32: {'BloodPressure': {69: {'Pregnancies': {7: 0.0,
                6: 0.0}},
              68: {'Pregnancies': {6: 0.0, 7: 0.0}}}},
            31: {'BloodPressure': {69: {'Pregnancies': {6: 0.0, 7: 0.0}},
              68: {'Pregnancies': {7: 0.0, 6: 0.0}}}}}}}},
        27.8: {'SkinThickness': {31: {'DiabetesPedigreeFunction': {0.527: {'BloodPressure': {68: {'Pregnancies': {6: 0.0,
                7: 0.0}},
              69: {'Pregnancies': {6: 0.0, 7: 0.0}}}},
            1.5270000000000001: {'BloodPressure': {68: {'Pregnancies': {6: 0.0,
                7: 1.0}},
              69: {'Pregnancies': {6: 0.0, 7: 0.0}}}}}},
          32: 0}}}},
      121: {'DiabetesPedigreeFunction': {0.527: {'BMI': {28.8: {'BloodPressure': {69: {'SkinThickness': {31: {'Pregnancies': {6: 0.0,
                7: 1.0}},
              32: {'Pregnancies': {6: 0.0, 7: 0.0}}}},
            68: {'SkinThickness': {31: 0.0, 32: 0.0}}}},
          27.8: 0}},
        1.5270000000000001: {'Pregnancies': {6: {'BMI': {28.8: {'SkinThickness': {31: 1,
              32: {'BloodPressure': {69: 0.0, 68: 1.0}}}},
            27.8: {'BMI': {27.8: 0.0}}}},
          7: 1}}}}}}}},
  128: {'BMI': {28.8: {'Pregnancies': {6: {'Age': {28: {'SkinThickness': {31: {'DiabetesPedigreeFunction': {0.527: {'Insulin': {120: {'BloodPressure': {68: 1.0,
                69: 0.0}},
              121: {'Insulin': {121: 0.0}}}},
            1.5270000000000001: {'Insulin': {121: {'BloodPressure': {68: 1.0,
                69: 0.0}},
              120: {'BloodPressure': {69: 1.0, 68: 0.0}}}}}},
          32: {'BloodPressure': {69: {'Insulin': {121: {'DiabetesPedigreeFunction': {0.527: 0.0,
                1.5270000000000001: 0.0}},
              120: {'DiabetesPedigreeFunction': {1.5270000000000001: 1.0,
                0.527: 1.0}}}},
            68: {'DiabetesPedigreeFunction': {1.5270000000000001: {'Insulin': {120: 0.0,
                121: 1.0}},
              0.527: 1}}}}}},
        29: {'BloodPressure': {69: {'DiabetesPedigreeFunction': {0.527: {'SkinThickness': {31: {'Insulin': {120: 1.0,
                121: 1.0}},
              32: {'Insulin': {120: 0.0, 121: 1.0}}}},
            1.5270000000000001: {'Insulin': {120: {'SkinThickness': {31: 1.0,
                32: 1.0}},
              121: {'SkinThickness': {31: 1.0, 32: 0.0}}}}}},
          68: {'SkinThickness': {31: {'Insulin': {121: {'DiabetesPedigreeFunction': {0.527: 1.0,
                1.5270000000000001: 0.0}},
              120: {'DiabetesPedigreeFunction': {1.5270000000000001: 1.0,
                0.527: 1.0}}}},
            32: 1}}}}}},
      7: {'DiabetesPedigreeFunction': {0.527: {'BloodPressure': {69: {'SkinThickness': {32: {'Insulin': {120: 1.0,
              121: 1.0}},
            31: {'Insulin': {120: 1.0, 121: 0.0}}}},
          68: 1}},
        1.5270000000000001: {'SkinThickness': {31: 1,
          32: {'Insulin': {121: {'BloodPressure': {68: 1.0, 69: 1.0}},
            120: 1}}}}}}}},
    27.8: {'SkinThickness': {31: {'DiabetesPedigreeFunction': {0.527: {'Pregnancies': {6: {'Insulin': {120: {'BloodPressure': {69: {'Age': {29: 0.0,
                28: 0.0}},
              68: {'Age': {28: 1.0, 29: 0.0}}}},
            121: {'Age': {28: 0, 29: {'BloodPressure': {68: 0.0, 69: 1.0}}}}}},
          7: {'Insulin': {121: 1,
            120: {'BloodPressure': {68: 0.0, 69: 0.0}}}}}},
        1.5270000000000001: {'Pregnancies': {6: 0,
          7: {'Insulin': {121: 0.0, 120: 0.0}}}}}},
      32: {'Insulin': {120: {'DiabetesPedigreeFunction': {1.5270000000000001: 1.0,
          0.527: 0.0}},
        121: 1}}}}}}}}

#### Contoh Hasil Rule Adaboost

##### Stump

In [16]:
adatree[0][0][0]

{'Glucose': {127: 0.0, 128: 1.0}}

##### Bobot

In [17]:
adatree[0][0][1]

1.0889875752939042